# import

In [46]:
import torch
import torch.nn as nn
from vilt.modules import heads, objectives
import vilt.modules.vision_transformer as vit
import torch.nn.functional as F
import random
from typing import OrderedDict
import os
import pandas as pd
import numpy as np
from vilt.transforms import pixelbert_transform
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
from torch.utils.data import DataLoader
import gc
import torch.optim as optim
from torch.optim import lr_scheduler
from collections import defaultdict
import wandb

from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold


# config

In [47]:


class config:
    debug = False
    sensor_only = False
    crop_name = "total"

    
    exp_name = "vilt"
    seed = 101
    batch_size = 4096  # this is a desired batch size; pl trainer will accumulate gradients when per step batch is smaller.
    train_batch_size = 32
    valid_batch_size = 4
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    n_fold = 5

    model_name = "sensorViLOnlyTransformerSS" #仅图片
    # model_name = "sensorOnlyViLTransformerSS"  #仅vilt传感器
    # model_name = "sensorViLTransformerSS"  #vilt图像+传感器
    # model_name = "DNNF1"  #DNNF1图像+传感器
    # model_name = "DNNF1PictureOnly"  #DNNF1图像
    # model_name = "DNNF1SensorOnly"  #DNNF1传感器

    # model_name = "DNNF2"  #DNNF2图像+传感器
    # model_name = "DNNF2PictureOnly"  #DNNF1图像
    # model_name = "DNNF2SensorOnly"  #DNNF1传感器
    # wandb 
    # wandb_name = "vilt|大豆|290图像加传感器"
    # wandb_name = "vilt|大豆|290仅传感器"
    wandb_name = "vilt|大豆|290仅图片"

    # wandb_name = "DNNF1|大豆|290图像加传感器"
    # wandb_name = "DNNF1|大豆|290仅图像"
    # wandb_name = "DNNF1|大豆|290仅传感器"
    
    # wandb_name = "DNNF2|大豆|290图像加传感器"
    # wandb_name = "DNNF2|大豆|290仅图像"
    # wandb_name = "DNNF2|大豆|290仅传感器"
    

    # Image setting
    train_transform_keys = ["pixelbert"]
    val_transform_keys = ["pixelbert"]
    img_size = 384
    max_image_len = -1
    patch_size = 32
    draw_false_image = 1
    image_only = False

    # Sensor
    # senser_input_num = 11 # 翔冠的传感器参数
    senser_input_num = 19 # 天航的传感器参数
    
    # Text Setting
    vqav2_label_size = 3129
    max_text_len = 40
    tokenizer = "bert-base-uncased"
    vocab_size = 30522 # vocabulary词汇数量
    whole_word_masking = False
    mlm_prob = 0.15
    draw_false_text = 0

    # Transformer Setting
    vit = "vit_base_patch32_384"
    hidden_size = 768  # 嵌入向量大小
    num_heads = 12
    num_layers = 12
    mlp_ratio = 4
    drop_rate = 0.1

    # Optimizer Setting
    optim_type = "adamw"
    learning_rate = 1e-3 #0.0015#2e-3 #
    weight_decay = 1e-4 # 0.01 ->1e-4
    decay_power = 1
    max_epoch = 50
    max_steps = 25000
    warmup_steps = 2500
    end_lr = 0
    lr_mult = 1  # multiply lr for downstream heads
    # T_max = 8000/train_batch_size*max_epoch 
    T_max = 4632/train_batch_size*max_epoch 

    # Downstream Setting
    get_recall_metric = False


    # below params varies with the environment
    data_root = ""
    log_dir = "result"
    per_gpu_batchsize = 0  # you should define this manually with per_gpu_batch_size=#
    num_gpus = 1
    num_nodes = 1
    load_path = "weights/vilt_200k_mlm_itm.ckpt"
    # load_path = "save_model_dict.pt"
    num_workers = 1
    precision = 16

# config = vars(config)
# config = dict(config)
config

if config.debug:
    config.max_epoch = 5

In [48]:
def setup_seed(seed):

    torch.manual_seed(seed)  # 为CPU设置随机种子
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    # torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
    torch.cuda.manual_seed_all(seed)  # 为所有GPU设置随机种子
    #os.environ['PYTHONHASHSEED'] = str(seed)
    
setup_seed(config.seed)

# wandb

In [49]:
# os.environ["WANDB_MODE"] = 'dryrun' # 离线模式
try:
    # wandb.log(key="*******") # if debug
    wandb.login() # storage in ~/.netrc file
    anonymous = None
except:
    anonymous = "must"
    print('\nGet your W&B access token from here: https://wandb.ai/authorize\n')


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


# 数据

In [50]:
def fetch_df(crop_name):
    if crop_name == "soybean":
        df_tianhang = pd.read_csv("/home/junsheng/ViLT/data/290-tianhang-soybean.csv")
        df_tianhang['image_path'] = df_tianhang['pic_key'].map(lambda x:os.path.join('/home/junsheng/data/tianhang_soybean',x.split('/')[-1]))
    elif crop_name == "rice":
        df_tianhang = pd.read_csv("/home/junsheng/ViLT/data/290-tianhang-rice.csv")
        df_tianhang['image_path'] = df_tianhang['pic_key'].map(lambda x:os.path.join('/home/junsheng/data/tianhang_rice',x.split('/')[-1]))
    elif crop_name == "corn":
        
        df_tianhang = pd.read_csv("/home/junsheng/ViLT/data/290-tianhang-corn.csv")
        df_tianhang['image_path'] = df_tianhang['pic_key'].map(lambda x:os.path.join('/home/junsheng/data/tianhang_corn',x.split('/')[-1]))
    elif crop_name == "total":
        df_tianhang_soybean = pd.read_csv("/home/junsheng/ViLT/data/290-tianhang-soybean.csv")
        df_tianhang_soybean['image_path'] = df_tianhang_soybean['pic_key'].map(lambda x:os.path.join('/home/junsheng/data/tianhang_soybean',x.split('/')[-1]))

        df_tianhang_rice = pd.read_csv("/home/junsheng/ViLT/data/290-tianhang-rice.csv")
        df_tianhang_rice['image_path'] = df_tianhang_rice['pic_key'].map(lambda x:os.path.join('/home/junsheng/data/tianhang_rice',x.split('/')[-1]))

        df_tianhang_corn = pd.read_csv("/home/junsheng/ViLT/data/290-tianhang-corn.csv")
        df_tianhang_corn['image_path'] = df_tianhang_corn['pic_key'].map(lambda x:os.path.join('/home/junsheng/data/tianhang_corn',x.split('/')[-1]))

        df_tianhang = pd.concat([df_tianhang_soybean,df_tianhang_rice,df_tianhang_corn])
    df_tianhang['label'] = df_tianhang['LAI']
    df_tianhang = df_tianhang.dropna()
    df_tianhang = df_tianhang.reset_index()
    # print(df_tianhang.shape)
    number_title = []
    # 归一化数值列
    recorder = {}
    for title in df_tianhang:
        # print(df_xiangguan[title].head())
        if title == 'raw_label':
            continue
        if df_tianhang[title].dtype != "object":
            
            number_title.append(title)
            x_min = df_tianhang[title].min()
            x_max = df_tianhang[title].max()
            # print(x_min,x_max)
            recorder[title] = (x_min,x_max)
            df_tianhang[title] = df_tianhang[title].map(lambda x:(x-x_min)/(x_max - x_min))
    # print(number_title)
    # print(recorder)

    # 选择传感器列
    # if crop_name=="corn":
    #     config.senser_input_num = 17
    #     tianhang_sensor = ['co2', 'stemp', 'stemp2', 'stemp3', 'stemp5', 'shumi', 'shumi2', 'shumi3', 'shumi5', 'humi', 'pm10', 'pm25', 'press', 'solar', 'temp', 'wind_d', 'wind_sp']
    # else:
    #     tianhang_sensor = ['co2', 'stemp', 'stemp2', 'stemp3', 'stemp4', 'stemp5', 'shumi', 'shumi2', 'shumi3', 'shumi4', 'shumi5', 'humi', 'pm10', 'pm25', 'press', 'solar', 'temp', 'wind_d', 'wind_sp']
    tianhang_sensor = ['co2', 'stemp', 'stemp2', 'stemp3', 'stemp5', 'shumi', 'shumi2', 'shumi3', 'shumi5', 'humi', 'pm10', 'pm25', 'press', 'solar', 'temp', 'wind_d', 'wind_sp']
    df_tianhang['sensor'] = df_tianhang[tianhang_sensor].values.tolist()
    # print("input dim:",len(tianhang_sensor))
    
    # 筛选仅传感器信息
    if config.sensor_only:
    # del df_tianhang['pic_key']
        df_tianhang.drop_duplicates(subset=['pic_key'],inplace=True,ignore_index=True)
    # print("*********************df shape:",df_tianhang.shape)
    
    # debug 特判
    df=df_tianhang
    if config.debug:
        df = df[:100]
    return df

# fetch_df(config.crop_name)

In [72]:
fetch_df('soybean').shape

(2658, 29)

create folds

In [51]:
def creat_folds(df):
    skf = StratifiedKFold(n_splits=config.n_fold, shuffle=True, random_state=config.seed)  
    for fold, (train_idx, val_idx) in enumerate(skf.split(df,df.date)):
        df.loc[val_idx, 'fold'] = fold
    print(df.groupby(['fold'])['label'].count())   
    return df 


# dataset


In [52]:
myTransforms = transforms.Compose([
    transforms.Resize((config.img_size,config.img_size)),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.4870, 0.5287, 0.4776],
    std=[0.1639, 0.1735, 0.1617],
)
])

def load_img(path):
    img =  Image.open(path).convert('RGB')
    img = myTransforms(img)
    return img

class BuildDataset(torch.utils.data.Dataset):
    def __init__(self, df, label=True, transforms=None):
        self.df         = df
        self.label      = label
        self.sensors = df['sensor'].tolist()
        self.img_paths  = df['image_path'].tolist()   
        if self.label:
            self.labels = df['label'].tolist()
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path  = self.img_paths[index]
        img = load_img(img_path)
        sensor = self.sensors[index]
        sensor = torch.tensor(sensor).unsqueeze(0) #[1,n]
        if self.label:
            label = self.labels[index]
            return torch.tensor(img).to(torch.float), torch.tensor(sensor).to(torch.float),torch.tensor(label).to(torch.float)
        else:
            return torch.tensor(img).to(torch.float), torch.tensor(sensor).to(torch.float)

# dataloader

In [53]:
def fetch_dataloader(fold:int,df):
    train_df = df.query("fold!=@fold").reset_index(drop=True)

    valid_df = df.query("fold==@fold").reset_index(drop=True)
    print("train_df.shape:",train_df.shape)
    print("valid_df.shape:",valid_df.shape)

    train_data  = BuildDataset(df=train_df,label=True)
    valid_data = BuildDataset(df=valid_df,label=True)

    train_loader = DataLoader(train_data, batch_size=config.train_batch_size,shuffle=True)
    valid_loader = DataLoader(valid_data, batch_size=config.valid_batch_size,shuffle=False)
    # test_loader = DataLoader(test_data, batch_size=config.test_batch_size,shuffle=False)
    return train_loader,valid_loader

def fetch_dataloader_ubiquatous():
    train_df = pd.concat((fetch_df('soybean'),fetch_df('rice')),axis=0,join='inner').reset_index(drop=True)

    valid_df = fetch_df('corn').reset_index(drop=True)
    print("train_df.shape:",train_df.shape)
    print("valid_df.shape:",valid_df.shape)

    train_data  = BuildDataset(df=train_df,label=True)
    valid_data = BuildDataset(df=valid_df,label=True)

    train_loader = DataLoader(train_data, batch_size=config.train_batch_size,shuffle=True)
    valid_loader = DataLoader(valid_data, batch_size=config.valid_batch_size,shuffle=False)
    # test_loader = DataLoader(test_data, batch_size=config.test_batch_size,shuffle=False)
    return train_loader,valid_loader



计算图像均值标准差

In [54]:
def get_mean_std_value(loader):
    '''
    求数据集的均值和标准差
    :param loader:
    :return:
    '''
    data_sum,data_squared_sum,num_batches = 0,0,0
       
    pbar = tqdm(enumerate(loader), total=len(loader), desc='caculating ')    
    # for data,sensor,label  in loader:
    for step,(data,sensor,label)  in pbar:
        # data: [batch_size,channels,height,width]
        # 计算dim=0,2,3维度的均值和，dim=1为通道数量，不用参与计算
        # data_sum += torch.mean(data,dim=[0,2,3])    # [batch_size,channels,height,width]
        data_sum += torch.mean(data,dim=[0,2,3])    # [batch_size,height,width,channels]
        # 计算dim=0,2,3维度的平方均值和，dim=1为通道数量，不用参与计算
        # data_squared_sum += torch.mean(data**2,dim=[0,2,3])  # [batch_size,channels,height,width]
        data_squared_sum += torch.mean(data**2,dim=[0,2,3])  # [batch_size,height,width,channels]
        # 统计batch的数量
        num_batches += 1

       
    # 计算均值
    mean = data_sum/num_batches
    # 计算标准差
    std = (data_squared_sum/num_batches - mean**2)**0.5
    return mean,std
# df = fetch_df(config.crop_name)
# df = creat_folds(df)
# train_loader,_ = fetch_dataloader(fold=0,df=df)
# mean,std = get_mean_std_value(train_loader)
# print('mean = {},std = {}'.format(mean,std))

# model

## VILT

sensorViLOnlyTransformerSS-仅vit

In [55]:
class sensorViLOnlyTransformerSS(nn.Module):

    def __init__(self, sensor_class_n, output_class_n):
        super().__init__()
        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)
        self.transformer = getattr(vit, config.vit)(
            pretrained=True, config=vars(config)
        )
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()
        self.pooler = heads.Pooler(config.hidden_size)
        self.classifier = nn.Linear(config.hidden_size, output_class_n)

    def infer(
        self,
        batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,
    ):

        if image_embeds is None and image_masks is None:
            img = batch["image"].to(config.device)

            (
                image_embeds,  # torch.Size([1, 217, 768])
                image_masks,  # torch.Size([1, 217])
                patch_index,
                image_labels,
            ) = self.transformer.visual_embed(
                img,
                max_image_len=config.max_image_len,
                mask_it=mask_image,
            )
        else:
            patch_index, image_labels = (
                None,
                None,
            )
        # 用embedding对数据输入预处理，降低维度
        image_embeds = image_embeds + self.token_type_embeddings(
            torch.full_like(image_masks, image_token_type_idx)
        )
        # sensor_masks = batch['sensor_masks'] # 序列数量
        batch_size = img.shape[0]
        sensor_masks = torch.ones(batch_size, 1).to(config.device)  # 序列数量
        image_masks = image_masks.to(config.device)
        co_embeds = image_embeds
        co_masks = image_masks

        x = co_embeds.to(config.device)  # torch.Size([1, 145, 768])

        for i, blk in enumerate(self.transformer.blocks):
            blk = blk.to(config.device)
            x, _attn = blk(x, mask=co_masks)  # co_masks = torch.Size([1, 211])

        x = self.transformer.norm(x)  # torch.Size([1, 240, 768])
        image_feats = x
        cls_feats = self.pooler(x)  # torch.Size([1, 768])
        # cls_feats = self.dense(x)
        # cls_feats = self.activation(cls_feats)
        cls_output = self.classifier(cls_feats)
        # m = nn.Softmax(dim=1)

        m = nn.Sigmoid()
        cls_output = m(cls_output)

        ret = {
            "image_feats": image_feats,
            "cls_feats": cls_feats,  # class features
            "raw_cls_feats": x[:, 0],
            "image_labels": image_labels,
            "image_masks": image_masks,

            "patch_index": patch_index,

            "cls_output": cls_output,
        }

        return ret

    def forward(self, batch):
        ret = dict()

        ret.update(self.infer(batch))
        return ret

sensorViLTransformerSS

In [56]:

class sensorViLTransformerSS(nn.Module):

    def __init__(self,sensor_class_n,output_class_n):
        super().__init__()
        self.sensor_linear = nn.Linear(sensor_class_n,config.hidden_size) 

        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)

        self.transformer = getattr(vit, config.vit)(
                pretrained=True, config=vars(config)
            )
       
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()


        self.pooler = heads.Pooler(config.hidden_size)

        # self.pooler.apply(objectives.init_weights)
        self.classifier = nn.Linear(config.hidden_size,output_class_n)

        hs = config.hidden_size


    def infer(
        self,
        batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,
    ):
        sensor = batch['sensor'].to(config.device)
        sensor_embeds = self.sensor_linear(sensor) # input[1,1,12]  output[1,1,768]
        

        if image_embeds is None and image_masks is None:
            img = batch["image"].to(config.device)
       
            (
                image_embeds, # torch.Size([1, 217, 768])
                image_masks, # torch.Size([1, 217])
                patch_index,
                image_labels,
            ) = self.transformer.visual_embed(
                img,
                max_image_len=config.max_image_len,
                mask_it=mask_image,
            )
        else:
            patch_index, image_labels = (
                None,
                None,
            )
        # 用embedding对数据输入预处理，降低维度
        image_embeds = image_embeds + self.token_type_embeddings(
                torch.full_like(image_masks, image_token_type_idx)
            )
        # sensor_masks = batch['sensor_masks'] # 序列数量
        batch_size = img.shape[0]
        sensor_masks = torch.ones(batch_size,1).to(config.device) # 序列数量
        image_masks = image_masks.to(config.device)
        co_embeds = torch.cat([sensor_embeds, image_embeds], dim=1) # torch.Size([1, 240, 768]) ->240=217+23
        co_masks = torch.cat([sensor_masks, image_masks], dim=1) # torch.Size([1, 240])

        x = co_embeds.to(config.device) # torch.Size([1, 211, 768])

        for i, blk in enumerate(self.transformer.blocks): 
            blk = blk.to(config.device)
            x, _attn = blk(x, mask=co_masks) # co_masks = torch.Size([1, 211])

        x = self.transformer.norm(x) # torch.Size([1, 240, 768])
        sensor_feats, image_feats = ( # torch.Size([1, 23, 768]),torch.Size([1, 217, 768])
            x[:, : sensor_embeds.shape[1]], # 后面字数输出23维
            x[:, sensor_embeds.shape[1] :], # 前面图片输出217维
        )
        cls_feats = self.pooler(x) # torch.Size([1, 768])
        # cls_feats = self.dense(x)
        # cls_feats = self.activation(cls_feats)
        cls_output = self.classifier(cls_feats)
        # m = nn.Softmax(dim=1)
        
        m = nn.Sigmoid()
        cls_output = m(cls_output)
        
        ret = {
           "sensor_feats":sensor_feats,
            "image_feats": image_feats,
            "cls_feats": cls_feats, # class features
            "raw_cls_feats": x[:, 0],
            "image_labels": image_labels,
            "image_masks": image_masks,
           
            "patch_index": patch_index,

            "cls_output":cls_output,
        }

        return ret

    def forward(self, batch):
        ret = dict()
        
        ret.update(self.infer(batch))
        return ret


sensorOnlyViLTransformerSS

In [57]:

class sensorOnlyViLTransformerSS(nn.Module):

    def __init__(self,sensor_class_n,output_class_n):
        super().__init__()
        self.sensor_linear = nn.Linear(sensor_class_n,config.hidden_size) 

        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)

        self.transformer = getattr(vit, config.vit)(
                pretrained=True, config=vars(config)
            )
       
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()


        self.pooler = heads.Pooler(config.hidden_size)

        # self.pooler.apply(objectives.init_weights)
        self.classifier = nn.Linear(config.hidden_size,output_class_n)

        hs = config.hidden_size


    def infer(
        self,
        batch,
        # mask_image=False,
        # image_token_type_idx=1,
        # image_embeds=None,
        # image_masks=None,
    ):
        sensor = batch['sensor'].to(config.device)
        sensor_embeds = self.sensor_linear(sensor) # input[1,1,12]  output[1,1,768]
        

        # if image_embeds is None and image_masks is None:
        #     img = batch["image"].to(config.device)
       
        #     (
        #         image_embeds, # torch.Size([1, 217, 768])
        #         image_masks, # torch.Size([1, 217])
        #         patch_index,
        #         image_labels,
        #     ) = self.transformer.visual_embed(
        #         img,
        #         max_image_len=config.max_image_len,
        #         mask_it=mask_image,
        #     )
        # else:
        #     patch_index, image_labels = (
        #         None,
        #         None,
        #     )
        # 用embedding对数据输入预处理，降低维度
        # image_embeds = image_embeds + self.token_type_embeddings(
        #         torch.full_like(image_masks, image_token_type_idx)
        #     )
        # sensor_masks = batch['sensor_masks'] # 序列数量
        # batch_size = img.shape[0]
        sensor_masks = torch.ones(sensor_embeds.shape[1],1).to(config.device) # 序列数量
        # image_masks = image_masks.to(config.device)
        # co_embeds = torch.cat([sensor_embeds, image_embeds], dim=1) # torch.Size([1, 240, 768]) ->240=217+23
        # co_masks = torch.cat([sensor_masks, image_masks], dim=1) # torch.Size([1, 240])
        co_embeds = sensor_embeds
        co_masks = sensor_masks

        x = co_embeds.to(config.device) # torch.Size([1, 1, 768])

        for i, blk in enumerate(self.transformer.blocks):
            blk = blk.to(config.device)
            x, _attn = blk(x, mask=co_masks)

        x = self.transformer.norm(x) # torch.Size([1, 240, 768])
        # sensor_feats, image_feats = ( # torch.Size([1, 23, 768]),torch.Size([1, 217, 768])
        #     x[:, : sensor_embeds.shape[1]], # 后面字数输出23维
        #     x[:, sensor_embeds.shape[1] :], # 前面图片输出217维
        # )
        cls_feats = self.pooler(x) # torch.Size([1, 768])
        # cls_feats = self.dense(x)
        # cls_feats = self.activation(cls_feats)
        cls_output = self.classifier(cls_feats)
        # m = nn.Softmax(dim=1)
        
        m = nn.Sigmoid()
        cls_output = m(cls_output)
        
        ret = {
        #    "sensor_feats":sensor_feats,
            # "image_feats": image_feats,
            "cls_feats": cls_feats, # class features
            "raw_cls_feats": x[:, 0],
            # "image_labels": image_labels,
            # "image_masks": image_masks,
           
            # "patch_index": patch_index,

            "cls_output":cls_output,
        }

        return ret

    def forward(self, batch):
        ret = dict()
        
        ret.update(self.infer(batch))
        return ret


sensorResnet50TransformerSS

In [58]:

class sensorResnet50TransformerSS(nn.Module):

    def __init__(self,sensor_class_n,output_class_n):
        super().__init__()
        self.sensor_linear = nn.Linear(sensor_class_n,config.hidden_size) 
        # resnet model
        resnet_model = pretrainedmodels.__dict__["resnet50"](
    num_classes=1000, pretrained='imagenet')
        features = list([resnet_model.conv1, resnet_model.bn1, resnet_model.relu, resnet_model.maxpool, resnet_model.layer1, resnet_model.layer2, resnet_model.layer3,resnet_model.layer4])
        conv = nn.Conv2d(2048, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
        bn = nn.BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        relu = nn.ReLU(inplace=True)


        self.resnet_features = nn.Sequential(*features,conv,bn,relu)

        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)

        self.transformer = getattr(vit, config.vit)(
                pretrained=True, config=vars(config)
            )
       
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()


        self.pooler = heads.Pooler(config.hidden_size)

        # self.pooler.apply(objectives.init_weights)
        self.classifier = nn.Linear(config.hidden_size,output_class_n)

        hs = config.hidden_size


    def infer(
        self,
        batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,
    ):
        sensor = batch['sensor'].to(config.device)
        sensor_embeds = self.sensor_linear(sensor) # input[1,1,12]  output[1,1,768]
        img = batch["image"].to(config.device)
        image_embeds = self.resnet_features(img) 
        image_embeds = image_embeds.flatten(2).transpose(1, 2)
        image_masks = torch.ones(image_embeds.shape[0],image_embeds.shape[1],dtype=torch.int64).to(config.device)

        # 用embedding对数据输入预处理，降低维度
        image_embeds = image_embeds + self.token_type_embeddings(
                torch.full_like(image_masks, image_token_type_idx)
            )
        # sensor_masks = batch['sensor_masks'] # 序列数量
        batch_size = img.shape[0]
        sensor_masks = torch.ones(batch_size,1).to(config.device) # 序列数量
        image_masks = image_masks.to(config.device)
        co_embeds = torch.cat([sensor_embeds, image_embeds], dim=1) # torch.Size([1, 240, 768]) ->240=217+23
        co_masks = torch.cat([sensor_masks, image_masks], dim=1) # torch.Size([1, 240])

        x = co_embeds.to(config.device) # torch.Size([1, 211, 768])

        for i, blk in enumerate(self.transformer.blocks): 
            blk = blk.to(config.device)
            x, _attn = blk(x, mask=co_masks) # co_masks = torch.Size([1, 211])

        x = self.transformer.norm(x) # torch.Size([1, 240, 768])
        sensor_feats, image_feats = ( # torch.Size([1, 23, 768]),torch.Size([1, 217, 768])
            x[:, : sensor_embeds.shape[1]], # 后面字数输出23维
            x[:, sensor_embeds.shape[1] :], # 前面图片输出217维
        )
        cls_feats = self.pooler(x) # torch.Size([1, 768])
        # cls_feats = self.dense(x)
        # cls_feats = self.activation(cls_feats)
        cls_output = self.classifier(cls_feats)
        # m = nn.Softmax(dim=1)
        
        m = nn.Sigmoid()
        cls_output = m(cls_output)
        
        ret = {
           "sensor_feats":sensor_feats,
            "image_feats": image_feats,
            "cls_feats": cls_feats, # class features
            "raw_cls_feats": x[:, 0],
            "image_masks": image_masks,
           

            "cls_output":cls_output,
        }

        return ret

    def forward(self, batch):
        ret = dict()
        
        ret.update(self.infer(batch))
        return ret


sensorResnet101TransformerSS

In [59]:

class sensorResnet101TransformerSS(nn.Module):

    def __init__(self,sensor_class_n,output_class_n):
        super().__init__()
        self.sensor_linear = nn.Linear(sensor_class_n,config.hidden_size) 
        # resnet model
        resnet_model = pretrainedmodels.__dict__["resnet101"](
    num_classes=1000, pretrained='imagenet')
        features = list([resnet_model.conv1, resnet_model.bn1, resnet_model.relu, resnet_model.maxpool, resnet_model.layer1, resnet_model.layer2, resnet_model.layer3,resnet_model.layer4])
        conv = nn.Conv2d(2048, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
        bn = nn.BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        relu = nn.ReLU(inplace=True)


        self.resnet_features = nn.Sequential(*features,conv,bn,relu)

        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)

        self.transformer = getattr(vit, config.vit)(
                pretrained=True, config=vars(config)
            )
       
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()


        self.pooler = heads.Pooler(config.hidden_size)

        # self.pooler.apply(objectives.init_weights)
        self.classifier = nn.Linear(config.hidden_size,output_class_n)

        hs = config.hidden_size


    def infer(
        self,
        batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,
    ):
        sensor = batch['sensor'].to(config.device)
        sensor_embeds = self.sensor_linear(sensor) # input[1,1,12]  output[1,1,768]
        img = batch["image"].to(config.device)
        image_embeds = self.resnet_features(img) 
        image_embeds = image_embeds.flatten(2).transpose(1, 2)
        image_masks = torch.ones(image_embeds.shape[0],image_embeds.shape[1],dtype=torch.int64).to(config.device)

        # 用embedding对数据输入预处理，降低维度
        image_embeds = image_embeds + self.token_type_embeddings(
                torch.full_like(image_masks, image_token_type_idx)
            )
        # sensor_masks = batch['sensor_masks'] # 序列数量
        batch_size = img.shape[0]
        sensor_masks = torch.ones(batch_size,1).to(config.device) # 序列数量
        image_masks = image_masks.to(config.device)
        co_embeds = torch.cat([sensor_embeds, image_embeds], dim=1) # torch.Size([1, 240, 768]) ->240=217+23
        co_masks = torch.cat([sensor_masks, image_masks], dim=1) # torch.Size([1, 240])

        x = co_embeds.to(config.device) # torch.Size([1, 211, 768])

        for i, blk in enumerate(self.transformer.blocks): 
            blk = blk.to(config.device)
            x, _attn = blk(x, mask=co_masks) # co_masks = torch.Size([1, 211])

        x = self.transformer.norm(x) # torch.Size([1, 240, 768])
        sensor_feats, image_feats = ( # torch.Size([1, 23, 768]),torch.Size([1, 217, 768])
            x[:, : sensor_embeds.shape[1]], # 后面字数输出23维
            x[:, sensor_embeds.shape[1] :], # 前面图片输出217维
        )
        cls_feats = self.pooler(x) # torch.Size([1, 768])
        # cls_feats = self.dense(x)
        # cls_feats = self.activation(cls_feats)
        cls_output = self.classifier(cls_feats)
        # m = nn.Softmax(dim=1)
        
        m = nn.Sigmoid()
        cls_output = m(cls_output)
        
        ret = {
           "sensor_feats":sensor_feats,
            "image_feats": image_feats,
            "cls_feats": cls_feats, # class features
            "raw_cls_feats": x[:, 0],
            "image_masks": image_masks,
           

            "cls_output":cls_output,
        }

        return ret

    def forward(self, batch):
        ret = dict()
        
        ret.update(self.infer(batch))
        return ret


## DNNF1

dnnf1 图片加传感器

In [60]:
class DNNF1(torch.nn.Module):
    
 
    def __init__(self,sensor_nums):
        super(DNNF1,self).__init__()
        self.sensor_linear = torch.nn.Linear(sensor_nums,768)
        
        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)

        self.transformer = getattr(vit, config.vit)(
                pretrained=True, config=vars(config)
            )
       
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()


        self.pooler = heads.Pooler(config.hidden_size)


        # DNNF1结构
        self.linear1=torch.nn.Linear(768+768,64)
        self.relu=torch.nn.ReLU()
        self.linear2=torch.nn.Linear(64,128)
        self.relu2=torch.nn.ReLU()
        self.linear3=torch.nn.Linear(128,256)
        self.relu3=torch.nn.ReLU()
        self.linear4=torch.nn.Linear(256,512)
        self.relu4=torch.nn.ReLU()
        self.linear5=torch.nn.Linear(512,512)
        self.relu5=torch.nn.ReLU()
        self.linear6=torch.nn.Linear(512,1024)
        self.relu6=torch.nn.ReLU()
        self.linear7=torch.nn.Linear(1024,1)


    def forward(self,batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,):
        sensor_input = batch['sensor'].to(config.device)
        sensor_feats = self.sensor_linear(sensor_input)

        if image_embeds is None and image_masks is None:
            img = batch["image"].to(config.device) # torch.Size([1, 3, 384, 384])

            (
                image_embeds,  # torch.Size([1, 217, 768])
                image_masks,  # torch.Size([1, 217])
                patch_index,
                image_labels,
            ) = self.transformer.visual_embed(
                img,
                max_image_len=config.max_image_len,
                mask_it=mask_image,
            )
        else:
            patch_index, image_labels = (
                None,
                None,
            )
        # 用embedding对数据输入预处理，降低维度
        image_embeds = image_embeds + self.token_type_embeddings(
            torch.full_like(image_masks, image_token_type_idx)
        )
        image_masks = image_masks.to(config.device)
        co_embeds = image_embeds
        co_masks = image_masks

        x = co_embeds.to(config.device)  # torch.Size([1, 145, 768])

        for i, blk in enumerate(self.transformer.blocks):
            blk = blk.to(config.device)
            x, _attn = blk(x, mask=co_masks)  # co_masks = torch.Size([1, 211])

        x = self.transformer.norm(x)  # torch.Size([1, 240, 768])
        picture_feats = self.pooler(x)  # torch.Size([1, 768])#图像的特征数据
        sensor_feats = sensor_feats.squeeze(dim=1) #torch.Size([1, 1, 768])->[1,768]

        x = torch.cat([picture_feats, sensor_feats], dim=1)

        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        x = self.relu4(x)
        x = self.linear5(x)
        x = self.relu5(x)
        x = self.linear6(x)
        x = self.relu6(x)
        x = self.linear7(x)
        return {"cls_output":x}

DNNF1 picture only


In [61]:
class DNNF1PictureOnly(torch.nn.Module):
    
 
    def __init__(self,sensor_nums):
        super(DNNF1PictureOnly,self).__init__()
        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)

        self.transformer = getattr(vit, config.vit)(
                pretrained=True, config=vars(config)
            )
       
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()


        self.pooler = heads.Pooler(config.hidden_size)


        # DNNF1结构
        self.linear1=torch.nn.Linear(768,64)
        self.relu=torch.nn.ReLU()
        self.linear2=torch.nn.Linear(64,128)
        self.relu2=torch.nn.ReLU()
        self.linear3=torch.nn.Linear(128,256)
        self.relu3=torch.nn.ReLU()
        self.linear4=torch.nn.Linear(256,512)
        self.relu4=torch.nn.ReLU()
        self.linear5=torch.nn.Linear(512,512)
        self.relu5=torch.nn.ReLU()
        self.linear6=torch.nn.Linear(512,1024)
        self.relu6=torch.nn.ReLU()
        self.linear7=torch.nn.Linear(1024,1)


    def forward(self,batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,):

        if image_embeds is None and image_masks is None:
            img = batch["image"].to(config.device) # torch.Size([1, 3, 384, 384])

            (
                image_embeds,  # torch.Size([1, 217, 768])
                image_masks,  # torch.Size([1, 217])
                patch_index,
                image_labels,
            ) = self.transformer.visual_embed(
                img,
                max_image_len=config.max_image_len,
                mask_it=mask_image,
            )
        else:
            patch_index, image_labels = (
                None,
                None,
            )
        # 用embedding对数据输入预处理，降低维度
        image_embeds = image_embeds + self.token_type_embeddings(
            torch.full_like(image_masks, image_token_type_idx)
        )
        image_masks = image_masks.to(config.device)
        co_embeds = image_embeds
        co_masks = image_masks

        x = co_embeds.to(config.device)  # torch.Size([1, 145, 768])

        for i, blk in enumerate(self.transformer.blocks):
            blk = blk.to(config.device)
            x, _attn = blk(x, mask=co_masks)  # co_masks = torch.Size([1, 211])

        x = self.transformer.norm(x)  # torch.Size([1, 240, 768])
        picture_feats = self.pooler(x)  # torch.Size([1, 768])#图像的特征数据

        x = picture_feats

        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        x = self.relu4(x)
        x = self.linear5(x)
        x = self.relu5(x)
        x = self.linear6(x)
        x = self.relu6(x)
        x = self.linear7(x)
        return {"cls_output":x}

DNNF1 sensor only

In [62]:
class DNNF1SensorOnly(torch.nn.Module):
    
 
    def __init__(self,sensor_nums):
        super(DNNF1SensorOnly,self).__init__()
        self.sensor_linear = torch.nn.Linear(sensor_nums,768)

        # DNNF1结构
        self.linear1=torch.nn.Linear(768,64)
        self.relu=torch.nn.ReLU()
        self.linear2=torch.nn.Linear(64,128)
        self.relu2=torch.nn.ReLU()
        self.linear3=torch.nn.Linear(128,256)
        self.relu3=torch.nn.ReLU()
        self.linear4=torch.nn.Linear(256,512)
        self.relu4=torch.nn.ReLU()
        self.linear5=torch.nn.Linear(512,512)
        self.relu5=torch.nn.ReLU()
        self.linear6=torch.nn.Linear(512,1024)
        self.relu6=torch.nn.ReLU()
        self.linear7=torch.nn.Linear(1024,1)


    def forward(self,batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,):
        sensor_input = batch['sensor'].to(config.device)
        sensor_feats = self.sensor_linear(sensor_input)

        sensor_feats = sensor_feats.squeeze(dim=1) #torch.Size([1, 1, 768])->[1,768]

        x = sensor_feats

        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        x = self.relu4(x)
        x = self.linear5(x)
        x = self.relu5(x)
        x = self.linear6(x)
        x = self.relu6(x)
        x = self.linear7(x)
        return {"cls_output":x}

## DNNF2

dnnf2 图片加传感器

In [63]:
class DNNF2(torch.nn.Module):
    
 
    def __init__(self,sensor_nums):
        super(DNNF2,self).__init__()
        self.sensor_linear = torch.nn.Linear(sensor_nums,768)
        
        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)

        self.transformer = getattr(vit, config.vit)(
                pretrained=True, config=vars(config)
            )
       
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()


        self.pooler = heads.Pooler(config.hidden_size)


        # DNNF2结构
        self.linear1=torch.nn.Linear(64+64,512)
        self.relu=torch.nn.ReLU()
        self.linear2=torch.nn.Linear(512,1024)
        self.relu2=torch.nn.ReLU()
        self.linear3=torch.nn.Linear(1024,1024)
        self.relu3=torch.nn.ReLU()
        self.linear4=torch.nn.Linear(1024,1)

        self.sensor_linear1 = torch.nn.Linear(768,32)
        self.sensor_relu1=torch.nn.ReLU()
        self.sensor_linear2 = torch.nn.Linear(32,64)
        self.sensor_relu2=torch.nn.ReLU()

        self.picture_linear1 = torch.nn.Linear(768,32)
        self.picture_relu1 = torch.nn.ReLU()
        self.picture_linear2 = torch.nn.Linear(32,64)
        self.picture_relu2 = torch.nn.ReLU()


    def forward(self,batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,):
        sensor_input = batch['sensor'].to(config.device)
        sensor_feats = self.sensor_linear(sensor_input)

        if image_embeds is None and image_masks is None:
            img = batch["image"].to(config.device) # torch.Size([1, 3, 384, 384])

            (
                image_embeds,  # torch.Size([1, 217, 768])
                image_masks,  # torch.Size([1, 217])
                patch_index,
                image_labels,
            ) = self.transformer.visual_embed(
                img,
                max_image_len=config.max_image_len,
                mask_it=mask_image,
            )
        else:
            patch_index, image_labels = (
                None,
                None,
            )
        # 用embedding对数据输入预处理，降低维度
        image_embeds = image_embeds + self.token_type_embeddings(
            torch.full_like(image_masks, image_token_type_idx)
        )
        image_masks = image_masks.to(config.device)
        co_embeds = image_embeds
        co_masks = image_masks

        x = co_embeds.to(config.device)  # torch.Size([1, 145, 768])

        for i, blk in enumerate(self.transformer.blocks):
            blk = blk.to(config.device)
            x, _attn = blk(x, mask=co_masks)  # co_masks = torch.Size([1, 211])

        x = self.transformer.norm(x)  # torch.Size([1, 240, 768])
        picture_feats = self.pooler(x)  # torch.Size([1, 768])#图像的特征数据
        sensor_feats = sensor_feats.squeeze(dim=1) #torch.Size([1, 1, 768])->[1,768]

        sensor_feats = self.sensor_linear1(sensor_feats)
        sensor_feats = self.sensor_relu1(sensor_feats)
        sensor_feats = self.sensor_linear2(sensor_feats)
        sensor_feats = self.sensor_relu2(sensor_feats)

        picture_feats = self.picture_linear1(picture_feats)
        picture_feats = self.picture_relu1(picture_feats)
        picture_feats = self.picture_linear2(picture_feats)
        picture_feats = self.picture_relu2(picture_feats)
        x = torch.cat([picture_feats, sensor_feats], dim=1)


        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        return {"cls_output":x}

DNNF2 picture only

In [64]:
class DNNF2PictureOnly(torch.nn.Module):
    
 
    def __init__(self,sensor_nums):
        super(DNNF2PictureOnly,self).__init__()
        self.sensor_linear = torch.nn.Linear(sensor_nums,768)
        
        self.token_type_embeddings = nn.Embedding(2, config.hidden_size)
        self.token_type_embeddings.apply(objectives.init_weights)

        self.transformer = getattr(vit, config.vit)(
                pretrained=True, config=vars(config)
            )
       
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()


        self.pooler = heads.Pooler(config.hidden_size)


        # DNNF2结构
        self.linear1=torch.nn.Linear(64,512)
        self.relu=torch.nn.ReLU()
        self.linear2=torch.nn.Linear(512,1024)
        self.relu2=torch.nn.ReLU()
        self.linear3=torch.nn.Linear(1024,1024)
        self.relu3=torch.nn.ReLU()
        self.linear4=torch.nn.Linear(1024,1)


        self.picture_linear1 = torch.nn.Linear(768,32)
        self.picture_relu1 = torch.nn.ReLU()
        self.picture_linear2 = torch.nn.Linear(32,64)
        self.picture_relu2 = torch.nn.ReLU()


    def forward(self,batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,):

        if image_embeds is None and image_masks is None:
            img = batch["image"].to(config.device) # torch.Size([1, 3, 384, 384])

            (
                image_embeds,  # torch.Size([1, 217, 768])
                image_masks,  # torch.Size([1, 217])
                patch_index,
                image_labels,
            ) = self.transformer.visual_embed(
                img,
                max_image_len=config.max_image_len,
                mask_it=mask_image,
            )
        else:
            patch_index, image_labels = (
                None,
                None,
            )
        # 用embedding对数据输入预处理，降低维度
        image_embeds = image_embeds + self.token_type_embeddings(
            torch.full_like(image_masks, image_token_type_idx)
        )
        image_masks = image_masks.to(config.device)
        co_embeds = image_embeds
        co_masks = image_masks

        x = co_embeds.to(config.device)  # torch.Size([1, 145, 768])

        for i, blk in enumerate(self.transformer.blocks):
            blk = blk.to(config.device)
            x, _attn = blk(x, mask=co_masks)  # co_masks = torch.Size([1, 211])

        x = self.transformer.norm(x)  # torch.Size([1, 240, 768])
        picture_feats = self.pooler(x)  # torch.Size([1, 768])#图像的特征数据

        picture_feats = self.picture_linear1(picture_feats)
        picture_feats = self.picture_relu1(picture_feats)
        picture_feats = self.picture_linear2(picture_feats)
        picture_feats = self.picture_relu2(picture_feats)
        
        x = picture_feats

        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        return {"cls_output":x}

DNNF2 sensor only


In [65]:
class DNNF2SensorOnly(torch.nn.Module):
    
 
    def __init__(self,sensor_nums):
        super(DNNF2SensorOnly,self).__init__()
        self.sensor_linear = torch.nn.Linear(sensor_nums,768)


        # DNNF2结构
        self.linear1=torch.nn.Linear(64,512)
        self.relu=torch.nn.ReLU()
        self.linear2=torch.nn.Linear(512,1024)
        self.relu2=torch.nn.ReLU()
        self.linear3=torch.nn.Linear(1024,1024)
        self.relu3=torch.nn.ReLU()
        self.linear4=torch.nn.Linear(1024,1)

        self.sensor_linear1 = torch.nn.Linear(768,32)
        self.sensor_relu1=torch.nn.ReLU()
        self.sensor_linear2 = torch.nn.Linear(32,64)
        self.sensor_relu2=torch.nn.ReLU()



    def forward(self,batch,
        mask_image=False,
        image_token_type_idx=1,
        image_embeds=None,
        image_masks=None,):
        sensor_input = batch['sensor'].to(config.device)
        sensor_feats = self.sensor_linear(sensor_input)

        sensor_feats = sensor_feats.squeeze(dim=1) #torch.Size([1, 1, 768])->[1,768]

        sensor_feats = self.sensor_linear1(sensor_feats)
        sensor_feats = self.sensor_relu1(sensor_feats)
        sensor_feats = self.sensor_linear2(sensor_feats)
        sensor_feats = self.sensor_relu2(sensor_feats)


        
        x = sensor_feats


        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        return {"cls_output":x}

## model build

In [66]:
import pretrainedmodels
from efficientnet_pytorch import EfficientNet

def build_model(model_name: str,pre_train):
    if model_name[:6] == "resnet50":
        model = pretrainedmodels.__dict__[config.model_name](
            num_classes=1000, pretrained='imagenet')
        dim_feats = model.last_linear.in_features  # =2048
        nb_classes = 1
        model.last_linear = nn.Linear(dim_feats, nb_classes)
        return model
    if model_name == "se_resnet50":
        model = pretrainedmodels.__dict__[config.model_name](
            num_classes=1000, pretrained='imagenet')
        model.last_linear = nn.Linear(204800, 1,bias=True)
        return model
    if model_name == "efficientnet-b4": # efficient net
        # refer:https://github.com/lukemelas/EfficientNet-PyTorch#example-classification
        nb_classes = 1
        if pre_train:
            model = EfficientNet.from_pretrained(config.model_name)# 'efficientnet-b4'
        else:
            model = EfficientNet.from_name(config.model_name)# 'efficientnet-b4'
        model._fc = nn.Linear(1792, nb_classes)
        return model

    if model_name == "sensorOnlyViLTransformerSS": #仅传感器
        model = sensorOnlyViLTransformerSS(sensor_class_n= config.senser_input_num,output_class_n = 1)
        return model
    if model_name == "sensorViLOnlyTransformerSS": # 仅vit图像
        model = sensorViLOnlyTransformerSS(sensor_class_n= config.senser_input_num,output_class_n = 1)
        return model
        
    if model_name == "sensorResnet50TransformerSS":
        model = sensorResnet50TransformerSS(sensor_class_n= config.senser_input_num,output_class_n = 1)
        return model
    if model_name == "sensorResnet101TransformerSS":
        model = sensorResnet101TransformerSS(sensor_class_n= config.senser_input_num,output_class_n = 1)
        return model

    if model_name == "sensorViLTransformerSS":
        model = sensorViLTransformerSS(sensor_class_n= config.senser_input_num,output_class_n = 1)
        return model

    if model_name == "DNNF1":
        model = DNNF1(sensor_nums=config.senser_input_num)
        return model
    if model_name == "DNNF1PictureOnly":
        model = DNNF1PictureOnly(sensor_nums=config.senser_input_num)
        return model
    if model_name == "DNNF1SensorOnly":
        model = DNNF1SensorOnly(sensor_nums=config.senser_input_num)
        return model
        
    if model_name == "DNNF2":
        model = DNNF2(sensor_nums=config.senser_input_num)
        return model
    if model_name == "DNNF2PictureOnly":
        model = DNNF2PictureOnly(sensor_nums=config.senser_input_num)
        return model
    if model_name == "DNNF2SensorOnly":
        model = DNNF2SensorOnly(sensor_nums=config.senser_input_num)
        return model


# for i,m in enumerate(model.modules()):
#     print(i,m)

test

In [67]:

# sensor = torch.rand(config.senser_input_num)
# # sensor = torch.ones(config.senser_input_num)
# print(sensor)
# sensor =  torch.tensor(sensor).unsqueeze(0).unsqueeze(0) # torch.Size([1, 1, 3])
# batch = {}
# batch['sensor'] = sensor
# batch['image'] = "/home/junsheng/data/xiangguan/pic/xiangguanD4-2021-05-24-10-00-25.jpeg"
# model(batch)

# 损失函数

In [68]:
criterion = F.mse_loss #均方误差损失函数
criterion_mae = nn.L1Loss()

# train one epoch

In [69]:



def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    dataset_size = 0
    running_loss = 0.0
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    for step, (img, sensor,label) in pbar:         
        # img = img.to(device, dtype=torch.float)
        # sensor  = sensor.to(device, dtype=torch.float)
        # label  = label.to(device, dtype=torch.float)
        batch_size = img.size(0)
        
        batch = {"image":img,"sensor":sensor}

        y_pred = model(batch)
        label = label.to(config.device).unsqueeze(1)
        loss = criterion(y_pred['cls_output'], label)
        
        #一坨优化
        optimizer.zero_grad()#每一次反向传播之前都要归零梯度
        loss.backward()      #反向传播
        optimizer.step()     #固定写法
        scheduler.step()
     
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        epoch_loss = running_loss / dataset_size
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_mem=f'{mem:0.2f} GB')

    
        
        
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss

# valid one epoch

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, optimizer):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    running_loss_mae = 0.0
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for step, (img, sensor,label) in pbar:               
        
        
        batch_size = img.size(0)
        batch = {"image":img,"sensor":sensor}

        y_pred  = model(batch)
        label = label.to(config.device).unsqueeze(1)

        loss = criterion(y_pred['cls_output'], label)
        loss_mae = criterion_mae(y_pred['cls_output'], label)

        running_loss += (loss.item() * batch_size)
        running_loss_mae += (loss_mae.item() * batch_size)

        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        epoch_loss_mae = running_loss_mae / dataset_size
        
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(valid_loss=f'{epoch_loss:0.4f}',
        valid_loss_mae=f'{epoch_loss_mae:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_memory=f'{mem:0.2f} GB')
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss,epoch_loss_mae#MSE，MAE

# train

In [ ]:

def run_training(model, optimizer, scheduler, device, num_epochs,train_loader,valid_loader):
     # init wandb
    run = wandb.init(project="vilt",
                    config={k: v for k, v in dict(vars(config)).items() if '__' not in k},
                    # config={k: v for k, v in dict(config).items() if '__' not in k},
                    anonymous=anonymous,
                    # name=f"vilt|fold-{config.valid_fold}",
                    name=config.wandb_name,
                    # group=config.wandb_group,
                    )
    wandb.watch(model, log_freq=100)

    best_loss = 9999
    best_valid_loss = 9999
    history = defaultdict(list)
    if torch.cuda.is_available():
        print("cuda: {}\n".format(torch.cuda.get_device_name()))
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        print(f'Epoch {epoch}/{num_epochs}', end='')
        train_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=device, epoch=epoch)
        val_loss,val_loss_mae = valid_one_epoch(model,valid_loader,device=device,optimizer=optimizer)
        history['Train Loss'].append(train_loss)
        history['Valid Loss'].append(val_loss)
        history['Valid Loss MAE'].append(val_loss_mae)

        wandb.log({"Train Loss": train_loss,
                    "Valid Loss": val_loss,
                    "Valid Loss MAE": val_loss_mae,
                "lr": scheduler.get_last_lr()[0]
                })
        if best_valid_loss > val_loss:
            best_valid_loss = val_loss
            # model_file_path = os.path.join(wandb.run.dir,"epoch-{}-{}.bin".format(epoch,wandb.run.id))
            # model_file_path = os.path.join(wandb.run.dir,"epoch-best.bin")
            # run.summary["Best Epoch"] = epoch
            # torch.save(model.state_dict(), model_file_path)
            # print("model save to", model_file_path)
               
    os.system("cp /home/junsheng/ViLT/my_vilt_tianhang_soybean.ipynb {}".format(wandb.run.dir))
    run.finish()
    return model, history

run train

In [ ]:
def run(crop_name:str,model_name:str,wandb_name:str,sensor_only:bool):
    config.model_name = model_name
    config.wandb_name = wandb_name
    config.sensor_only = sensor_only

    df = fetch_df(crop_name)
    df = creat_folds(df)

    # train_loader,valid_loader = fetch_dataloader(fold=0,df=df)
    train_loader,valid_loader = fetch_dataloader_ubiquatous()

    model = build_model(config.model_name,True)
    model.to(config.device)
    print(config.device)
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=config.T_max, 
                                                    eta_min=5e-8)
    model, history = run_training(model, optimizer, scheduler,device=config.device,num_epochs=config.max_epoch,train_loader=train_loader,valid_loader=valid_loader)




tasks

In [ ]:
soybean_task = [
    # ********************vilt*****************
    {
        "crop_name":"soybean",
        "model_name":"sensorViLOnlyTransformerSS",
        "wandb_name":"vilt|大豆|290仅图片",
        "sensor_only":True,
    },
    {
        "crop_name":"soybean",
        "model_name":"sensorOnlyViLTransformerSS",
        "wandb_name":"vilt|大豆|290仅传感器",
        "sensor_only":True,
    },
    {
        "crop_name":"soybean",
        "model_name":"sensorViLTransformerSS",
        "wandb_name":"vilt|大豆|290图像加传感器",
        "sensor_only":False,
    },

    # ********************DNNF1*****************
    {
        "crop_name":"soybean",
        "model_name":"DNNF1",
        "wandb_name":"DNNF1|大豆|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"soybean",
        "model_name":"DNNF1PictureOnly",
        "wandb_name":"DNNF1|大豆|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"soybean",
        "model_name":"DNNF1SensorOnly",
        "wandb_name":"DNNF1|大豆|290仅传感器",
        "sensor_only":True,
    },
    # ********************DNNF2*****************
    {
        "crop_name":"soybean",
        "model_name":"DNNF2",
        "wandb_name":"DNNF2|大豆|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"soybean",
        "model_name":"DNNF2PictureOnly",
        "wandb_name":"DNNF2|大豆|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"soybean",
        "model_name":"DNNF2SensorOnly",
        "wandb_name":"DNNF2|大豆|290仅传感器",
        "sensor_only":True,
    },

]

corn_task = [
    # ********************vilt*****************
    {
        "crop_name":"corn",
        "model_name":"sensorViLOnlyTransformerSS",
        "wandb_name":"vilt|玉米|290仅图片",
        "sensor_only":True,
    },
    {
        "crop_name":"corn",
        "model_name":"sensorOnlyViLTransformerSS",
        "wandb_name":"vilt|玉米|290仅传感器",
        "sensor_only":True,
    },
    {
        "crop_name":"corn",
        "model_name":"sensorViLTransformerSS",
        "wandb_name":"vilt|玉米|290图像加传感器",
        "sensor_only":False,
    },

    # ********************DNNF1*****************
    {
        "crop_name":"corn",
        "model_name":"DNNF1",
        "wandb_name":"DNNF1|玉米|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"corn",
        "model_name":"DNNF1PictureOnly",
        "wandb_name":"DNNF1|玉米|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"corn",
        "model_name":"DNNF1SensorOnly",
        "wandb_name":"DNNF1|玉米|290仅传感器",
        "sensor_only":True,
    },
    # ********************DNNF2*****************
    {
        "crop_name":"corn",
        "model_name":"DNNF2",
        "wandb_name":"DNNF2|玉米|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"corn",
        "model_name":"DNNF2PictureOnly",
        "wandb_name":"DNNF2|玉米|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"corn",
        "model_name":"DNNF2SensorOnly",
        "wandb_name":"DNNF2|玉米|290仅传感器",
        "sensor_only":True,
    },

]

rice_task = [
    # ********************vilt*****************
    {
        "crop_name":"rice",
        "model_name":"sensorViLOnlyTransformerSS",
        "wandb_name":"vilt|水稻|290仅图片",
        "sensor_only":True,
    },
    {
        "crop_name":"rice",
        "model_name":"sensorOnlyViLTransformerSS",
        "wandb_name":"vilt|水稻|290仅传感器",
        "sensor_only":True,
    },
    {
        "crop_name":"rice",
        "model_name":"sensorViLTransformerSS",
        "wandb_name":"vilt|水稻|290图像加传感器",
        "sensor_only":False,
    },

    # ********************DNNF1*****************
    {
        "crop_name":"rice",
        "model_name":"DNNF1",
        "wandb_name":"DNNF1|水稻|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"rice",
        "model_name":"DNNF1PictureOnly",
        "wandb_name":"DNNF1|水稻|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"rice",
        "model_name":"DNNF1SensorOnly",
        "wandb_name":"DNNF1|水稻|290仅传感器",
        "sensor_only":True,
    },
    # ********************DNNF2*****************
    {
        "crop_name":"rice",
        "model_name":"DNNF2",
        "wandb_name":"DNNF2|水稻|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"rice",
        "model_name":"DNNF2PictureOnly",
        "wandb_name":"DNNF2|水稻|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"rice",
        "model_name":"DNNF2SensorOnly",
        "wandb_name":"DNNF2|水稻|290仅传感器",
        "sensor_only":True,
    },

]


total_task = [
   # ********************vilt*****************
    {
        "crop_name":"total",
        "model_name":"sensorViLOnlyTransformerSS",
        "wandb_name":"vilt|普适|290仅图片",
        "sensor_only":True,
    },
    {
        "crop_name":"total",
        "model_name":"sensorOnlyViLTransformerSS",
        "wandb_name":"vilt|普适|290仅传感器",
        "sensor_only":True,
    },
    {
        "crop_name":"total",
        "model_name":"sensorViLTransformerSS",
        "wandb_name":"vilt|普适|290图像加传感器",
        "sensor_only":False,
    },

    # ********************DNNF1*****************
    {
        "crop_name":"total",
        "model_name":"DNNF1",
        "wandb_name":"DNNF1|普适|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"total",
        "model_name":"DNNF1PictureOnly",
        "wandb_name":"DNNF1|普适|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"total",
        "model_name":"DNNF1SensorOnly",
        "wandb_name":"DNNF1|普适|290仅传感器",
        "sensor_only":True,
    },
    # ********************DNNF2*****************
    {
        "crop_name":"total",
        "model_name":"DNNF2",
        "wandb_name":"DNNF2|普适|290图像加传感器",
        "sensor_only":False,
    },
    {
        "crop_name":"total",
        "model_name":"DNNF2PictureOnly",
        "wandb_name":"DNNF2|普适|290仅图像",
        "sensor_only":True,
    },
    {
        "crop_name":"total",
        "model_name":"DNNF2SensorOnly",
        "wandb_name":"DNNF2|普适|290仅传感器",
        "sensor_only":True,
    },


]

run tast

In [ ]:
# for i in range(3): # 0 1 2
# for i in [6,7,8]: # 0 1 2
#     config.learning_rate = 5e-5
#     run(total_task[i]["crop_name"],total_task[i]["model_name"],total_task[i]["wandb_name"],total_task[i]["sensor_only"])
config.senser_input_num = 17
ubiquatous_task =     {
        "crop_name":"total",
        "model_name":"sensorViLTransformerSS",
        "wandb_name":"vilt|普适|290图像加传感器->水稻",
        "sensor_only":False,
    }
config.learning_rate = 5e-5
run(ubiquatous_task["crop_name"],ubiquatous_task["model_name"],ubiquatous_task["wandb_name"],ubiquatous_task["sensor_only"])

    

fold
0.0    927
1.0    927
2.0    926
3.0    926
4.0    926
Name: label, dtype: int64
train_df.shape: (1974, 29)
valid_df.shape: (2658, 29)


/tmp/ipykernel_1917183/2459798531.py:41: RuntimeWarning: invalid value encountered in double_scalars
  df_tianhang[title] = df_tianhang[title].map(lambda x:(x-x_min)/(x_max - x_min))
No pretrained weights exist or were found for this model. Using random initialization.


cuda:1


cuda: NVIDIA GeForce RTX 3090

Epoch 1/50

Train :   0%|          | 0/62 [00:00<?, ?it/s]/tmp/ipykernel_1917183/3683523438.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(img).to(torch.float), torch.tensor(sensor).to(torch.float),torch.tensor(label).to(torch.float)
Train :   0%|          | 0/62 [00:01<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [1, 17] at entry 0 and [1, 19] at entry 1